In [9]:
# export $(cat .env | xargs)

In [10]:
%reload_ext autoreload
%autoreload 2

In [11]:
import sys
print(sys.executable)

/home/mle-user/env_recsys_start/bin/python3.10


In [20]:
import requests

import pandas as pd

In [13]:
final_recommendations = pd.read_parquet("final_recommendations.parquet")

In [14]:
final_recommendations

,score,user_id,item_id,count,max_pop_genres,mean_pop_genres,target,cb_score,rank
9,0.005479,0,39089632,57450,29665807.0,21060964.5,0,0.124063,1
5,0.006123,0,35505245,95523,35121554.0,29243470.0,0,0.115644,2
7,0.005562,0,9047114,57059,29665807.0,21060964.5,0,0.086290,3
8,0.005551,0,33308946,49991,29665807.0,15204585.5,0,0.077790,4
1,0.006914,0,732401,63119,29665807.0,21060964.5,0,0.073271,5
...,...,...,...,...,...,...,...,...,...
13723713,0.165555,1374582,49961817,68698,51490799.0,37759042.0,0,0.152265,6
13723716,0.151150,1374582,56920237,49212,51490799.0,37759042.0,0,0.112275,7
13723710,0.189659,1374582,45499814,78563,51490799.0,37759042.0,0,0.098346,8
13723717,0.149480,1374582,51241318,78533,3145110.0,3145110.0,0,0.066418,9


In [15]:
top_popular = pd.read_parquet("top_popular.parquet")

In [16]:
top_popular

,item_id,count,id,type,name
0,53404,110025,53404,track,Smells Like Teen Spirit
1,33311009,101075,33311009,track,Believer
2,178529,100866,178529,track,Numb
3,35505245,95523,35505245,track,I Got Love
4,24692821,84153,24692821,track,Way Down We Go
...,...,...,...,...,...
999656,88117091,1,88117091,track,Bessou Rassoul
999657,96542446,1,96542446,track,Urotsukodouji
999658,89358500,1,89358500,track,2 минуты
999659,99247670,1,99247670,track,Твои волшебные мечты


In [22]:
als_similar = pd.read_parquet("als_similar.parquet")

In [23]:
als_similar

,score,item_id,rec_item_id
0,1.000000,26,26
1,0.964777,26,1704328
2,0.964728,26,10851471
3,0.963184,26,2163978
4,0.959817,26,560452
...,...,...,...
10999995,0.971645,101521819,70626416
10999996,0.968042,101521819,85659885
10999997,0.962324,101521819,78094473
10999998,0.962320,101521819,87856142


In [ ]:
recommendations_url = "http://127.0.0.1:8000"
features_store_url = "http://127.0.0.1:8010"
events_store_url = "http://127.0.0.1:8020"

# Test 1

In [ ]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 2, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs) 

{'recs': [29456722, 50684233, 50685843]}


In [ ]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 99999999999999, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs) 

{'recs': [53404, 33311009, 178529]}


# Test 2

In [ ]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"item_id": 26, "k": 3}

resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
if resp.status_code == 200:
    similar_items = resp.json()
else:
    similar_items = None
    print(f"status code: {resp.status_code}")
    
print(similar_items) 

{'rec_item_id': [26, 1704328, 10851471], 'score': [0.9999998807907104, 0.9647767543792725, 0.9647282958030701]}


# Test 3

In [ ]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

{'result': 'ok'}


In [ ]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

{'events': [17245]}


# Test 4

In [64]:
user_id = 8888888888888888
event_item_ids = [26, 38, 135, 138]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})
                         
params = {"user_id": user_id, 'k': 5}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)

{'recs': [135, 38, 138, 15016378, 953663, 18871672, 784988, 18013001, 11284973, 18982805, 18560183, 5301796, 39279955, 2261742, 1895617]}


# Test 5

In [69]:
user_id = 8888888888888888
event_item_ids = [26, 38, 135, 138]


for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})

In [70]:
params = {"user_id": 8888888888888888, 'k': 10}
resp_offline = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
resp_online = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
resp_blended = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)

recs_offline = resp_offline.json()["recs"]
recs_online = resp_online.json()["recs"]
recs_blended = resp_blended.json()["recs"]

print(recs_offline)
print(recs_online)
print(recs_blended)

[53404, 33311009, 178529, 35505245, 24692821, 795836, 6705392, 32947997, 37384, 45499814]
[135, 38, 138, 15016378, 953663, 18871672, 784988, 18013001, 667787, 781976, 7737254, 686489, 18297547, 11284973, 18982805, 18560183, 18982807, 65295441, 55266917, 53973566, 5301796, 39279955, 42958618, 2261742, 1895617, 37849144, 30163817, 17916395, 10059297, 32810492]
[135, 53404, 38, 33311009, 138, 178529, 15016378, 35505245, 953663, 24692821]
